In [1]:
!pip install PyMuPDF

     |████████████████████████████████| 6.3MB 5.0MB/s 


In [3]:
import pandas as pd
import numpy as np
import pickle
import fitz
import re
import statistics 
from statistics import mode

In [15]:
def getFontBold(page, start, end):
  fontSizes = []
  for i in range(start, end):
    index = page[i]

    tempHTMLString = index.getText("html")
    imgOccuranceList = [i.start() for i in re.finditer('<img', tempHTMLString)]
    for i in range(len(imgOccuranceList)):
      tempHTMLString = tempHTMLString.replace(tempHTMLString[tempHTMLString.find('<img'): tempHTMLString.find('>',tempHTMLString.find('<img')) + 1], '<>')

    for line in tempHTMLString.splitlines():
      if (line.find('<b>') != -1):
        individualFont = line[line.find('font-size:') + 10: line.find('pt',line.find('font-size:'))]
        individualFont = float(individualFont)
        fontSizes.append(individualFont)

  fontSizes = list(set(fontSizes))
  fontSizes.sort()
  return fontSizes

In [18]:
def getFont(page, start, end):
  fontSizes = []
  for i in range(start, end):
    index = page[i]

    tempHTMLString = index.getText("html")
    imgOccuranceList = [i.start() for i in re.finditer('<img', tempHTMLString)]
    for i in range(len(imgOccuranceList)):
      tempHTMLString = tempHTMLString.replace(tempHTMLString[tempHTMLString.find('<img'): tempHTMLString.find('>',tempHTMLString.find('<img')) + 1], '<>')
    
    for line in tempHTMLString.splitlines():
      if (line.find('div') == -1 and line.find('<>') == -1):
        individualFont = line[line.find('font-size:') + 10: line.find('pt',line.find('font-size:'))]
        individualFont = float(individualFont)
        fontSizes.append(individualFont)

  mostFrequentFont = mode(fontSizes)
  fontSizes = list(set(fontSizes))
  fontSizes.sort()
  fontSizes = [i for i in fontSizes if i >= mostFrequentFont]
  return fontSizes

In [21]:
def getLabels(page, start, end):
  lt=getFont(page,start,end)
  finalList = []  
  for i in range(start, end):
    individualFont = 0
    index = page[i]
    tempHTMLString = index.getText("html")
    imgOccuranceList = [i.start() for i in re.finditer('<img', tempHTMLString)]
    for i in range(len(imgOccuranceList)):
      tempHTMLString = tempHTMLString.replace(tempHTMLString[tempHTMLString.find('<img'): tempHTMLString.find('>',tempHTMLString.find('<img')) + 1], '<>')
    for line in tempHTMLString.splitlines():
      try:
        if line.find('div') != -1:  #Ignore the div tag
          continue
        
        if (line.find('div') == -1 and line.find('<>') == -1):
          individualText = line[line.find('>',line.find("<span ")) + 1: line.find('</span>')]
          individualFont = line[line.find('font-size:') + 10: line.find('pt',line.find('font-size:'))]
          styling = re.sub(line[line.find('>',line.find("<span ")) + 1: line.find('</span>')],'',line)
          #print(f)
          individualFont = float(individualFont)
          if ( individualFont == lt[len(lt) - 1]):
            print('Main Heading: ' + individualText)
            individualFont = 1
            finalList.append([individualText,styling,individualFont])
          else:
            if ( individualFont == lt[len(lt) - 2]):
              print('Heading: ' + individualText)
              individualFont = 2
              finalList.append([individualText,styling,individualFont])
            else:
              if ( individualFont == lt[len(lt) - 3]):
                if (line.find('<b>') != -1):
                  print('Sub Heading: ' + individualText)
                  individualFont = 3
                  finalList.append([individualText,styling,individualFont])
              else:
                if ( individualFont == lt[len(lt) - 4]):
                  print('Sub Sub Heading: ' + individualText)
                  individualFont = 4
                  finalList.append([individualText,styling,individualFont])
                else:
                  if ( styling[styling.find('font-family:') + 12: styling.find(';',styling.find('font-family:'))] == "UbuntuMono,monospace"):
                    print('Code ' + individualText)
                    individualFont = 5
                    finalList.append([individualText,styling,individualFont])
      except:
        continue    
  return finalList